In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymap3d as pm

In [2]:
df = pd.read_csv("/home/torsten/ms/results/with_gnss.csv", header=None, skiprows=[])
df

,0,1,2,3,4
0,1,0,0.000154,0.000412,0.000061
1,1,1,-0.996860,0.177555,-7.236810
2,2,0,-0.116728,-0.028926,-0.144100
3,2,1,0.545500,0.135718,-0.326219
4,2,2,-5.850210,-1.823070,-3.633040
...,...,...,...,...,...
20295,200,196,9.119240,-5.890200,-7.428120
20296,200,197,10.642000,-6.841700,-7.346440
20297,200,198,12.822900,-7.997830,-7.199360
20298,200,199,14.890500,-7.863380,-7.038390


In [3]:
# -33.9618,18.4596,137.171
lat0 = -33.9618
lon0 = 18.4596
h0 = 137.171

df_gnss = pd.read_csv("/home/torsten/ms/data/rosbag2_old_zoo_a_09_29/ublox.csv")
df_gnss

,sec,nanosec,lat,lon,alt,vel_n,vel_e,vel_d,h_acc
0,1695990717,339259638,-33.961781,18.459736,135.127,0.06,-0.02,0.01,4.333
1,1695990717,744454095,-33.961781,18.459736,135.205,0.00,0.00,-0.01,4.334
2,1695990718,139187619,-33.961781,18.459737,135.246,0.00,0.01,0.00,4.335
3,1695990718,542473933,-33.961781,18.459737,135.272,-0.03,0.01,0.00,4.336
4,1695990718,950614801,-33.961781,18.459737,135.273,0.02,-0.01,0.00,4.338
...,...,...,...,...,...,...,...,...,...
1062,1695991146,956316730,-33.961826,18.459754,132.007,-0.02,0.01,0.03,4.539
1063,1695991147,346157102,-33.961826,18.459755,131.991,-0.03,0.03,0.04,4.547
1064,1695991147,750519063,-33.961826,18.459755,132.039,0.00,0.02,0.03,4.555
1065,1695991148,146838381,-33.961826,18.459755,132.045,-0.03,0.01,0.03,4.563


In [4]:
# Sat image segments
segment_filenames = ["1.csv", "2.csv", "3.csv", "4.csv"]
segment_coords = []
segments_enu = []

for fn in segment_filenames:
    df_seg = pd.read_csv("~/ms/data/old_zoo_sat_paths/" + fn)
    segment_coords.append(df_seg.iloc[:, 0:2].values)

for seg in segment_coords:
    segments_enu.append(np.zeros(seg.shape))
    for i in range(seg.shape[0]):
        enu = pm.geodetic2enu(seg[i, 1], seg[i, 0], 102, lat0, lon0, h0)
        segments_enu[-1][i, 0] = enu[0]
        segments_enu[-1][i, 1] = enu[1]
    

[[ 18.45957323 -33.96181525]
 [ 18.45947641 -33.96179326]
 [ 18.4594103  -33.96178314]
 [ 18.45930496 -33.96177596]
 [ 18.45923416 -33.96178844]
 [ 18.45917958 -33.96181506]
 [ 18.45911336 -33.9618589 ]
 [ 18.45908441 -33.96188454]]
[[ 18.45957323 -33.96181525]
 [ 18.45947641 -33.96179326]
 [ 18.4594103  -33.96178314]
 [ 18.45930496 -33.96177596]
 [ 18.45923416 -33.96178844]
 [ 18.45917958 -33.96181506]
 [ 18.45911336 -33.9618589 ]
 [ 18.45908441 -33.96188454]]
[[ 18.45957323 -33.96181525]
 [ 18.45947641 -33.96179326]
 [ 18.4594103  -33.96178314]
 [ 18.45930496 -33.96177596]
 [ 18.45923416 -33.96178844]
 [ 18.45917958 -33.96181506]
 [ 18.45911336 -33.9618589 ]
 [ 18.45908441 -33.96188454]]
[[ 18.45957323 -33.96181525]
 [ 18.45947641 -33.96179326]
 [ 18.4594103  -33.96178314]
 [ 18.45930496 -33.96177596]
 [ 18.45923416 -33.96178844]
 [ 18.45917958 -33.96181506]
 [ 18.45911336 -33.9618589 ]
 [ 18.45908441 -33.96188454]]
[[ 18.45957323 -33.96181525]
 [ 18.45947641 -33.96179326]
 [ 18.4594

In [6]:
%matplotlib tk
start = 0

plt.figure()
plt.axis('equal')
plt.grid()
# Factor graph
for i in range(200):
    plt.plot(df.iloc[start:start+i+2, 2], df.iloc[start:start+i+2, 3], "-x", color="grey")
    plt.text(df.iloc[start+i+1, 2], df.iloc[start+i+1, 3], str(i+1))
    
    start+=i+2

# GNSS trail
for index, row in df_gnss.iterrows():
    if index % 10 != 0:
        continue
    enu = pm.geodetic2enu(row["lat"], row["lon"], row["alt"], lat0, lon0, h0)
    plt.plot(enu[0], enu[1], "-bx")
    plt.errorbar(enu[0], enu[1], row["h_acc"], row["h_acc"])

# Sat image segments
for seg in segments_enu:
    plt.plot(seg[:, 0], seg[:, 1], "-", color="blue")

plt.show()